In [ ]:
# scraper.py (Enhanced Version)

import os
import requests
import csv
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
API_KEY = os.getenv("GNEWS_API_KEY")
BASE_URL = "https://gnews.io/api/v4"

def fetch_news(query):
    """Fetches news based on a search query from the GNews API."""
    endpoint = "/search"
    
    # Parameters for the request
    params = {
        'q': query,
        'lang': 'en',
        'country': 'us',
        'max': 10,
        'sortby': 'publishedAt', # or 'relevance'
        'apikey': '7a1dbf56434669bf051c5894e3b0ec42'
    }

    try:
        response = requests.get(BASE_URL + endpoint, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get('articles', [])
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for query '{query}': {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        # Check if the API returned an error message in the JSON
        try:
            error_details = response.json()
            print("API Error:", error_details.get('errors'))
        except:
            pass # Can't parse JSON, just stick to the original exception
        return []


def save_articles_to_csv(articles, query):
    """Saves a list of articles to a CSV file."""
    if not articles:
        print("No articles to save.")
        return

    # Create a unique filename with a timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"news_{query.replace(' ', '_')}_{timestamp}.csv"

    # Define the headers for our CSV file
    headers = ['title', 'description', 'url', 'publishedAt', 'sourceName', 'sourceUrl']

    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            
            # Write the header row
            writer.writerow(headers)
            
            # Write the article data
            for article in articles:
                writer.writerow([
                    article.get('title'),
                    article.get('description'),
                    article.get('url'),
                    article.get('publishedAt'),
                    article.get('source', {}).get('name'), # Safely access nested 'name'
                    article.get('source', {}).get('url')   # Safely access nested 'url'
                ])
        print(f"Successfully saved {len(articles)} articles to {filename}")
    except IOError as e:
        print(f"Error writing to CSV file: {e}")

if __name__ == "__main__":
    search_query = "artificial intelligence" 
    
    print(f"Searching for news about: '{search_query}'")
    

    found_articles = fetch_news(search_query)

    if found_articles:
        save_articles_to_csv(found_articles, search_query)
    else:
        print("Could not find any articles for the given query.")

Searching for news about: 'artificial intelligence'
Successfully saved 10 articles to news_artificial_intelligence_20250621_114941.csv
